In [1]:
import os
import gc
import re
import sys
import math
import json
import time
import eli5
import lofo
import optuna
import random
import joblib
import pickle
import warnings
import difflib
import Levenshtein
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRanker
from glob import glob
from pathlib import Path
from unidecode import unidecode
import multiprocessing
from tqdm.auto import tqdm
from argparse import Namespace
import matplotlib.pyplot as plt
from BorutaShap import BorutaShap
from sklearn.metrics import f1_score, fbeta_score, roc_auc_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import haversine_distances
from lofo import LOFOImportance, Dataset, plot_importance
from sklearn.metrics import average_precision_score
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, StratifiedGroupKFold
from eli5.sklearn import PermutationImportance

from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore", module="lightgbm")

plt.rcParams["font.size"] = 13
sns.set_style("darkgrid")

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 600)

/home/alex/anaconda3/envs/ubiquant/lib/python3.9/site-packages/lofo/lofo_importance.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Config

In [2]:
CFG = Namespace(
    train = False,
    full = False,
    debug = False,
    optimize = False,
    select_features = True,
    load_params = False,
    selection_type = 'lofo', # feasible values: lofo, perm, shap, corr, gain
    test = False,
    folds = 0,
    seed = 42,
    pos_frac = 0,
    target = 'match',
    threshold = 0.5,
    train_path = 'train_dataset',
    model_dir = 'fsq_lgbm_models',
    es_rounds = 50
)

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(CFG.seed)

# Prepare data

## Load dataset utils

In [3]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']

def downcast_floats(df):
    floats = ['float32', 'float64']
    float_features = list(df.select_dtypes(include=floats).columns)
    for f in float_features:
        df[f] = df[f].astype('float16')
    return df
    
def load_dataset(label='train'):
    if CFG.full or CFG.folds:
        train_files = glob(os.path.join(CFG.train_path, "train_*.pkl"))
        valid_files = glob(os.path.join(CFG.train_path, "valid_*.pkl"))
        train_files = train_files + valid_files
    else:
        train_files = glob(os.path.join(CFG.train_path, f"{label}_*.pkl"))

    train = list()
    for filename in tqdm(train_files):
        # load data
        df = pd.read_pickle(filename)

        # set features
        features = list(df.select_dtypes(include=numerics).columns)
        features.remove(CFG.target)

        if CFG.debug:
            df = df.sample(n = 10000, random_state = CFG.seed)
            df = df.reset_index(drop = True)
        df = downcast_floats(df)
        train.append(df)

    train = pd.concat(train, axis=0, ignore_index=True)
    gc.collect()
    
    return train

## Add the rest matches

In [4]:
# if not CFG.full and not CFG.folds:
#     all_matches_files = glob(os.path.join(CFG.train_path, "all_matches_*.pkl"))
    
#     all_matches = list()
#     for filename in tqdm(all_matches_files):
#         df = pd.read_pickle(filename)
#         if CFG.debug:
#             df = df.sample(n = 10000, random_state = CFG.seed)
#             df = df.reset_index(drop = True)
#         df = downcast_floats(df)
#         all_matches.append(df)

#     all_matches = pd.concat(all_matches, axis=0, ignore_index=True)
    
    
# all_matches['label'] = 1
# all_matches = all_matches[all_matches['id'] != all_matches['match_id']]
# all_matches = all_matches[['id', 'match_id', 'label'] + features]

## Add matches to train and test dataset

In [5]:
# %%time

# all_matches_ids = set(all_matches['id'].unique())

# train_ids = set(train['id'].unique())
# all_matches_train_ids = list(train_ids.intersection(all_matches_ids))

# if not CFG.full and not CFG.folds:
#     valid_ids = set(valid['id'].unique())
#     all_matches_valid_ids = list(valid_ids.intersection(all_matches_ids))

# all_matches = all_matches.set_index('id')
# all_matches_train = all_matches.loc[all_matches_train_ids]
# all_matches_train = all_matches_train.reset_index()
# train = pd.concat([train, all_matches_train], axis=0, ignore_index=True)
# train = train.drop_duplicates(['id', 'match_id'])
# del train_ids, all_matches_ids, all_matches_train_ids

# if not CFG.full and not CFG.folds:
#     all_matches_valid = all_matches.loc[all_matches_valid_ids]
#     all_matches_valid = all_matches_valid.reset_index()
#     valid = pd.concat([valid, all_matches_valid], axis=0, ignore_index=True)
#     valid = valid.drop_duplicates(['id', 'match_id'])
#     del valid_ids, all_matches_valid_ids, all_matches_valid

# del all_matches, all_matches_train
# gc.collect()

## Increase fraction of positive targets

In [6]:
# %%time

# if CFG.pos_frac:
#     train_pos_index = train[train['label'] == 1].index
#     train_neg_index = train[train['label'] == 0].index
#     train_neg_index = np.random.choice(train_neg_index, size=int(len(train_pos_index)*((1-CFG.pos_frac)/CFG.pos_frac)))
#     train_pos_index = np.concatenate([train_pos_index, train_neg_index])
#     np.random.shuffle(train_pos_index)
#     train = train.loc[train_pos_index].reset_index(drop=True)
#     del train_pos_index, train_neg_index
#     gc.collect()

#     if not CFG.full and not CFG.folds:
#         valid_pos_index = valid[valid['label'] == 1].index
#         valid_neg_index = valid[valid['label'] == 0].index
#         valid_neg_index = np.random.choice(valid_neg_index, size=int(len(valid_pos_index)*((1-CFG.pos_frac)/CFG.pos_frac)))
#         valid_pos_index = np.concatenate([valid_pos_index, valid_neg_index])
#         np.random.shuffle(valid_pos_index)
#         valid = valid.loc[valid_pos_index].reset_index(drop=True)
#         del valid_pos_index, valid_neg_index
#         gc.collect() 

# Optimize with Optuna

In [8]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).

def objective(trial):
    dtrain = lgb.Dataset(train[features], label=train[CFG.target])
    dvalid = lgb.Dataset(valid[features], label=valid[CFG.target])

    param = {
        'seed': CFG.seed,
#         'device': 'gpu',
#         'gpu_platform_id': 0,
#         'gpu_device_id': 0,
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical("boosting_type", ['gbdt']),# 'dart', 'goss']),
        'force_col_wise': False, # Use only with CPU devices
        'subsample_for_bin': 300000, # Number of data that sampled to construct feature discrete bins; setting this 
                                     # to larger value will give better training result but may increase train time
        'n_estimators': 200, #trial.suggest_int('n_estimators', 300, 1000),      
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 3e-1),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256), # Max number of leaves in one tree
        'max_bin': trial.suggest_int('max_bin', 32, 255), # Max number of bins that feature values will be 
                                                           # bucketed in. small number of bins may reduce training 
                                                           # accuracy but may deal with overfitting
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0), # Randomly select a subset of features 
                                                                               # if feature_fraction < 1.0
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0), # Randomly select part of data without 
                                                                               # resampling if bagging_fraction < 1.0
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7), # Perform bagging at every k iteration
        'min_data_in_leaf': trial.suggest_int('min_child_samples', 5, 64), # Minimal number of data in one leaf
                                                                            # aliases: min_child_samples, 
        'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf', 1e-4, 1e-1), # Stop trying to split 
                                                                                               # leave if sum of it's
                                                                                               # hessian less than k
#         'cat_smooth': trial.suggest_float('cat_smooth', 10.0, 100.0), # this can reduce the effect of noises in 
#                                                                       # categorical features, especially for 
#                                                                       # categories with few data
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'auc')
    gbm = lgb.train(
        param, 
        dtrain, 
        valid_sets=[dvalid],
        callbacks = [lgb.log_evaluation(10), 
                     lgb.early_stopping(stopping_rounds=30)]
    )

    # Evaluation
    preds = gbm.predict(valid[features])
    roc_auc = roc_auc_score(valid[CFG.target], preds)
    return roc_auc


if CFG.optimize:
    train = load_dataset('train')
    valid = load_dataset('valid')
    
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="maximize"
    )
    study.optimize(objective, timeout=12*3600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
        
    # Save study to dataframe
    study_df = study.trials_dataframe()
    study_df.to_csv('additional_data/optuna_lgbm.csv')

# Set parameters

In [9]:
if CFG.load_params:
    lgb_params = pd.read_csv('additional_data/optuna_lgbm.csv')

    param_cols = [c for c in lgb_params.columns if c.startswith('params_')]
    lgb_params = lgb_params.sort_values('value', ascending=False)[param_cols].head(10)

    best_params = list()

    def param_to_set(row):
        row_dict = {k[7:]: v for k, v in row.items()}
        row_dict['seed'] = CFG.seed
        row_dict['objective'] = 'binary'
        row_dict['metric'] = 'auc'
        row_dict['n_estimators'] = 1500
        row_dict['verbose'] = -1
    #     row_dict['device'] = 'gpu'
    #     row_dict['gpu_platform_id'] = 0
    #     row_dict['gpu_device_id'] = 0
        best_params.append(row_dict)

    params = lgb_params.apply(param_to_set, axis=1)
else:
    params = {
         'seed': CFG.seed,
         'boosting_type': 'gbdt',
         'reg_alpha': 0,
         'reg_lambda': 10,
         'max_depth': 64,
         'num_leaves': 256,
         'learning_rate': 0.07,
         'n_estimators': 500,
         'min_child_samples': 20,
         'subsample': 0.5,
         'colsample_bytree': 0.5,
         'colsample_bynode': 1,
         'objective': 'binary',
         'metric': 'auc',
         'verbose': -1,
         'seed': CFG.seed,
         'subsample_freq': 1,
         'is_unbalance': False,
         'importance_type': 'gain'
    }

if CFG.test:
    params['n_estimators'] = 200
    
if CFG.select_features:
    # extract a sample of the data
    train = load_dataset('train')
    train = train.sample(frac=0.2, random_state=CFG.seed)
    valid = load_dataset('valid')
    valid = valid.sample(frac=0.2, random_state=CFG.seed)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

# Feature importance

## LOFO importance

In [ ]:
features_to_check = ['manhattan', 'euclidian', 'main_category_jaccard', 'main_category_overlap', 
                     'main_category_cosine', 'closest_city1', 'closest_city2', 'tfidf_trigram_name_cleaned',
                     'tfidf_trigram_full_address_cleaned', 'p1_manhattan_mean', 'p2_manhattan_mean',
                     'p1_manhattan_min', 'p2_manhattan_min', 'p1_manhattan_max',
                     'p2_manhattan_max', 'p1_manhattan_rank', 'p2_manhattan_rank',
                     'p1_euclidian_mean', 'p2_euclidian_mean', 'p1_euclidian_min',
                     'p2_euclidian_min', 'p1_euclidian_max', 'p2_euclidian_max',
                     'p1_euclidian_rank', 'p2_euclidian_rank']


if CFG.select_features and CFG.selection_type=='lofo':
    # define the validation scheme
    cv = KFold(n_splits=2)
    train = pd.concat([train, valid], ignore_index=True)
    del valid
    gc.collect()
    # define the binary target and the features
    dataset = lofo.Dataset(df=train, target=CFG.target, features=features_to_check)
    # define the validation scheme and scorer
    lofo_imp = lofo.LOFOImportance(dataset, scoring="roc_auc", cv=cv, model=lgb.LGBMClassifier(**params))
    # get the mean and standard deviation of the importances in pandas format
    importance_df = lofo_imp.get_importance()
    importance_df.to_csv('additional_data/importance_df.csv')
    # plot the means and standard deviations of the importances
    lofo.plot_importance(importance_df, figsize=(12, 20))

  0%|          | 0/25 [00:00<?, ?it/s]

## Permutation importance

In [11]:
if CFG.select_features and CFG.selection_type=='perm':   
    # fit model
    model=lgb.LGBMClassifier(**params)
    model.fit(train[features], train[CFG.target], eval_set=(valid[features], valid[CFG.target]))
    # get permutation importance
    perm = PermutationImportance(model, random_state=CFG.seed).fit(valid[features], valid[CFG.target])
    eli5.show_weights(perm, feature_names = features)

## SHAP importance

In [12]:
if CFG.select_features and CFG.selection_type=='perm':   
    train[features] = train[features].fillna(-9999)
    # fit model
    model=lgb.LGBMClassifier(**params)
    # calculate importance
    feature_selector = BorutaShap(importance_measure='shap', classification=True)
    feature_selector.fit(X=train[features], y=train[CFG.target], n_trials=50, sample=False, train_or_test = 'test', normalize=True, verbose=True)
    feature_selector.plot(which_features='all', figsize=(16,12))

## Gain importance

In [13]:
if CFG.select_features and CFG.selection_type=='gain':   
    train[features] = train[features].fillna(-9999)
    # fit model
    model=lgb.LGBMClassifier(**params)
    # calculate importance
    feature_selector = BorutaShap(importance_measure='gini', classification=True)
    feature_selector.fit(X=train[features], y=train[CFG.target], n_trials=50, sample=False, train_or_test = 'test', normalize=True, verbose=True)
    feature_selector.plot(which_features='all', figsize=(16,12))

## Check correlation between features

In [14]:
if CFG.select_features and CFG.selection_type=='corr':
    features_corr = train.fillna(0).corr()
    # transform to low triangle matrix
    for i in range(features_corr.shape[0]):
        for j in range(features_corr.shape[1]):
            if j >= i:
                features_corr.iloc[i, j] = 0
    # unstack
    features_corr = features_corr.abs().unstack()
    features_corr = features_corr.reset_index()
    # select features with corr > 0 and sort them 
    features_corr = features_corr[features_corr[0] > 0]
    features_corr = features_corr.sort_values(0, kind="quicksort", ascending=False)
    display(features_corr.head(100))

# Train and validation

## Utils for postprocessing and validation

In [18]:
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['p1'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['p1'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    id2poi = get_id2poi(input_df)
    poi2ids = get_poi2ids(input_df)
    for id_str, matches in zip(input_df['p1'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    
    return scores.mean()

def postprocess(df):
    id2match = dict(zip(df["p1"].values, df["matches"].str.split()))

    for match in df["matches"].values:
        match = match.split()
        if len(match) == 1:        
            continue

        base = match[0]
        for m in match[1:]:
            if not base in id2match[m]:
                id2match[m].append(base)
    df["matches"] = df["id"].map(id2match).map(" ".join)
    
    return df 

def get_matches(df, preds):
    match_id = df["p2"].values
    matches = []

    for df_id, pred, match_idx in tqdm(zip(df["p1"], preds, match_id), total=df.shape[0]):
        idx = np.round(pred)
        if pred == 1:
            matches.append(df_id + " " + match_idx)
        else:
            matches.append(df_id)
    
    df['matches'] = matches
    df = postprocess(df)
    
    return df[['p1', 'matches', 'point_of_interest']]

## Train models

In [19]:
def pickle_save(obj, filename):
    pickle.dump(obj, open(filename, 'wb'))

def pickle_load(filename):
    return pickle.load(open(filename, 'rb'))

models = list()

for idx, (train_label, val_label) in enumerate([('train', 'valid'), ('valid', 'train')]):
    train_df = load_dataset(train_label)
    val_df = load_dataset(val_label)

    cat_features = [#"main_category1", "main_category2",
                    #"closest_city1", "closest_city2",
                    "country1", "country2", 
                    "categories1", "categories2"]
    num_features = [x for x in train_df.columns
                    if x not in ['p1', 'p2', 'match', 'main_category1', 'main_category2'] + cat_features]
    target = "match"

    lgbmc = LGBMClassifier(**params)

    # Train
    lgbmc.fit(train_df[num_features + cat_features],
              train_df[target],
              categorical_feature=cat_features,
              eval_metric="auc",
              eval_set=(
                  val_df[num_features + cat_features], val_df[target]),
              callbacks = [lgb.log_evaluation(10), 
                           lgb.early_stopping(stopping_rounds=CFG.es_rounds)])

    # Save model
    pickle_save(lgbmc, f"{CFG.model_dir}/lgbmc_fold{idx}.pkl")

    # Predict
    val_df["predict_proba"] = lgbmc.predict_proba(val_df[num_features + cat_features])[:, 1]
    pickle_save(val_df["predict_proba"].to_numpy(), f"{CFG.model_dir}/lgbmc_outoffold{idx}.pkl")

    # Show MAP metrics
    print("MAP:", average_precision_score(val_df["match"], val_df["predict_proba"]))
    
    # Add POI column to validation dataset
    data_root = 'foursquare_location_matching'
    data = pd.read_csv(os.path.join(data_root, 'train.csv'))[['id', 'point_of_interest']]

    val_df = val_df.merge(data, how='left', left_on='p1', right_on='id')

    del data
    gc.collect()
    
    # Calculate Jaccard
    y_hat = np.where(val_df["predict_proba"] < CFG.threshold, 0, 1) 
    res = get_matches(val_df, y_hat)
    res = res.drop_duplicates()
    cv = get_score(res)
    print(f'Jaccard is {cv:.6f}')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
[10]	valid_0's auc: 0.991258
[20]	valid_0's auc: 0.992255
[30]	valid_0's auc: 0.993076
[40]	valid_0's auc: 0.993594
[50]	valid_0's auc: 0.994064
[60]	valid_0's auc: 0.994329
[70]	valid_0's auc: 0.99458
[80]	valid_0's auc: 0.99475
[90]	valid_0's auc: 0.994869
[100]	valid_0's auc: 0.994937
[110]	valid_0's auc: 0.994977
[120]	valid_0's auc: 0.995044
[130]	valid_0's auc: 0.995074
[140]	valid_0's auc: 0.995129
[150]	valid_0's auc: 0.995145
[160]	valid_0's auc: 0.995172
[170]	valid_0's auc: 0.995195
[180]	valid_0's auc: 0.995199
[190]	valid_0's auc: 0.995233
[200]	valid_0's auc: 0.995238
[210]	valid_0's auc: 0.995303
[220]	valid_0's auc: 0.995352
[230]	valid_0's auc: 0.995359
[240]	valid_0's auc: 0.99538
[250]	valid_0's auc: 0.995401
[260]	valid_0's auc: 0.995416
[270]	valid_0's auc: 0.995427
[280]	valid_0's auc: 0.995467
[290]	valid_0's auc: 0.995474
[300]	valid_0's auc: 0.99549
[310]	valid_0's auc: 0.995515
[320]	valid_0's auc: 0

  0%|          | 0/24182861 [00:00<?, ?it/s]

Jaccard is 0.876094


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
[10]	valid_0's auc: 0.991581
[20]	valid_0's auc: 0.992501
[30]	valid_0's auc: 0.993166
[40]	valid_0's auc: 0.993587
[50]	valid_0's auc: 0.993937
[60]	valid_0's auc: 0.994223
[70]	valid_0's auc: 0.99446
[80]	valid_0's auc: 0.994587
[90]	valid_0's auc: 0.994712
[100]	valid_0's auc: 0.994794
[110]	valid_0's auc: 0.994832
[120]	valid_0's auc: 0.994874
[130]	valid_0's auc: 0.994896
[140]	valid_0's auc: 0.994918
[150]	valid_0's auc: 0.994956
[160]	valid_0's auc: 0.99498
[170]	valid_0's auc: 0.994988
[180]	valid_0's auc: 0.995001
[190]	valid_0's auc: 0.995027
[200]	valid_0's auc: 0.995037
[210]	valid_0's auc: 0.995049
[220]	valid_0's auc: 0.995052
[230]	valid_0's auc: 0.995069
[240]	valid_0's auc: 0.995083
[250]	valid_0's auc: 0.995096
[260]	valid_0's auc: 0.995093
[270]	valid_0's auc: 0.995107
[280]	valid_0's auc: 0.995116
[290]	valid_0's auc: 0.995123
[300]	valid_0's auc: 0.995137
[310]	valid_0's auc: 0.99514
[320]	valid_0's auc: 

  0%|          | 0/24175388 [00:00<?, ?it/s]

Jaccard is 0.876023


# Plot importance

In [ ]:
def plot_importance(model):
    importance_df = pd.DataFrame(model.feature_importances_, 
                                 index=num_features + cat_features, 
                                 columns=['importance'])\
                        .sort_values("importance", ascending=False)

    plt.figure(figsize=(16, 8))
    plt.bar(importance_df.index, importance_df.importance)
    plt.grid()
    plt.xticks(rotation=90)
    plt.ylabel("importance")
    plt.tight_layout()
    plt.show()
    
# def plot_importances(models):
#     importance_df = pd.DataFrame(models[0].feature_importances_, 
#                                  index=num_features, 
#                                  columns=['importance'])\
#                         .sort_values("importance", ascending=False)

#     plt.figure(figsize=(16, 8))
#     plt.bar(importance_df.index, importance_df.importance)
#     plt.grid()
#     plt.xticks(rotation=90)
#     plt.ylabel("importance")
#     plt.tight_layout()
#     plt.show()
    
plot_importance(lgbmc)

In [ ]:
# Baseline
# AUC: 0.995502/0.99511
# IOU: 0.875165/0.874105
# LB: 0.882

# Add new features (closest_city, main_categories, manhattan, euclidian, TF-IDF of some text columns, etc.)
# AUC: 0.995776/0.995172
# IOU: 0.876292/0.874874
# LB: 

# Fix encoder bug, 
# AUC: 0.995784/0.995196
# IOU: 0.876094/0.876023
# LB: 